In [ ]:
import duckdb

%load_ext sql
conn = duckdb.connect("../rewards_dbt/rewards_db.duckdb")
%sql conn --alias duckdb

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

In [14]:
%%sql
select * exclude(rewards_receipt_item_list)
from
    core.receipts

Running query in 'duckdb'

id,bonus_points_earned,bonus_points_earned_reason,create_date,scanned_date,finished_date,modify_date,points_awarded_date,points_earned,purchase_date,purchase_item_count,rewards_receipt_status,total_spent,user_id
5ff1e1eb0a720f0523000575,500,"Receipt number 2 completed, bonus point schedule DEFAULT (5cefdcacf3693e0b50e83a36)",2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:36,2021-01-03 15:25:31,500.00,2021-01-03 00:00:00,5,FINISHED,26.00,5ff1e1eacfcf6c399c274ae6
5ff1e1bb0a720f052300056b,150,"Receipt number 5 completed, bonus point schedule DEFAULT (5cefdcacf3693e0b50e83a36)",2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,150.00,2021-01-02 15:24:43,2,FINISHED,11.00,5ff1e194b6a9d73a3a9f1052
5ff1e1f10a720f052300057a,5,All-receipts receipt bonus,2021-01-03 15:25:37,2021-01-03 15:25:37,None,2021-01-03 15:25:42,None,5.00,2021-01-03 00:00:00,1,REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b
5ff1e1ee0a7214ada100056f,5,All-receipts receipt bonus,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:39,2021-01-03 15:25:34,5.00,2021-01-03 00:00:00,4,FINISHED,28.00,5ff1e1eacfcf6c399c274ae6
5ff1e1d20a7214ada1000561,5,All-receipts receipt bonus,2021-01-03 15:25:06,2021-01-03 15:25:06,2021-01-03 15:25:11,2021-01-03 15:25:11,2021-01-03 15:25:06,5.00,2021-01-02 15:25:06,2,FINISHED,1.00,5ff1e194b6a9d73a3a9f1052
5ff1e1e40a7214ada1000566,750,"Receipt number 1 completed, bonus point schedule DEFAULT (5cefdcacf3693e0b50e83a36)",2021-01-03 15:25:24,2021-01-03 15:25:24,2021-01-03 15:25:25,2021-01-03 15:25:30,2021-01-03 15:25:25,750.00,2021-01-02 15:25:24,1,FINISHED,3.25,5ff1e1e4cfcf6c399c274ac3
5ff1e1cd0a720f052300056f,5,All-receipts receipt bonus,2021-01-03 15:25:01,2021-01-03 15:25:01,2021-01-03 15:25:02,2021-01-03 15:25:02,2021-01-03 15:25:02,5.00,2021-01-03 15:25:01,1,FINISHED,2.23,5ff1e194b6a9d73a3a9f1052
5ff1e1a40a720f0523000569,500,"Receipt number 2 completed, bonus point schedule DEFAULT (5cefdcacf3693e0b50e83a36)",2021-01-03 15:24:20,2021-01-03 15:24:20,2021-01-03 15:24:21,2021-01-03 15:24:21,2021-01-03 15:24:21,500.00,2020-12-27 00:00:00,1,FINISHED,10.00,5ff1e194b6a9d73a3a9f1052
5ff1e1ed0a7214ada100056e,5,All-receipts receipt bonus,2021-01-03 15:25:33,2021-01-03 15:25:33,2021-01-03 15:25:34,2021-01-03 15:25:38,2021-01-03 15:25:34,5.00,2021-01-03 00:00:00,5,FINISHED,20.00,5ff1e1eacfcf6c399c274ae6
5ff1e1eb0a7214ada100056b,250,"Receipt number 3 completed, bonus point schedule DEFAULT (5cefdcacf3693e0b50e83a36)",2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:36,2021-01-03 15:25:31,250.00,2021-01-03 00:00:00,3,FINISHED,20.00,5ff1e1eacfcf6c399c274ae6


In [12]:
%%sql
--select * exclude(rewards_receipt_item_list)
select
    rewards_receipt_status,
    count(0) as count
from
    core.receipts
group by
    1

Running query in 'duckdb'

rewards_receipt_status,count
FLAGGED,46
PENDING,50
FINISHED,518
SUBMITTED,434
REJECTED,71


In [17]:
%%sql
select
    rewards_receipt_status,
    round(avg(total_spent)) as avg_spend
from
    core.receipts
where
    rewards_receipt_status in ('FINISHED', 'REJECTED')
group by
    1


Running query in 'duckdb'

rewards_receipt_status,avg_spend
REJECTED,23.0
FINISHED,81.0


With the results above, we see that the average spend from receipts with `rewards_receipt_status` value of Accepted(Finished) and Rejected. This value comes with the assumption that a value of `FINISHED` is a synonum for `ACCEPTED`. This assumption needs to be verified with the product owner/business to verify. If this assumption is wrong, more logic will be necessary to produce an accurate value.

I also assume that the status values of flagged, pending, and submitted do not fall within the definition of accepted or rejected. This is also another clarification to follow up on.